<a href="https://colab.research.google.com/github/Bryan-Kan/fatec_PLN_Codes/blob/master/Aula8_Introdu%C3%A7%C3%A3oaMLparaPLN/Aula_8_Introdu%C3%A7%C3%A3o_a_ML_para_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 08 - Introdução a ML para PLN


In [1]:
# Passo 1 - Criar o Corpus

corpus = [
    ("Eu amo PLN", "positivo"),
    ("Eu odeio bugs", "negativo"),
    ("Amo resolver problemas", "positivo"),
    ("Odeio erros", "negativo"),
]

#passo 2 - processar o texto
import re
from collections import defaultdict, Counter

def preprocess_text(text):
  return re.findall(r'\b\w+\b',text.lower())

processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)

#Passo 3 - Calculando probabilidade

# A partir do corpus processado, o código calcula as probabilidades necessárias para a classificação.

# Inicializa um contador para as classes e para as palavras dentro de cada classe
class_counts = Counter()  # Contagem das classes (positivo, negativo)
word_counts = defaultdict(Counter)  # Contagem das palavras por classe
total_words = defaultdict(int)  # Contagem do total de palavras por classe

# Preenche as contagens a partir do corpus processado
for words, label in processed_corpus:
  class_counts[label] += 1  # Conta a ocorrência de cada classe (positivo ou negativo)
  for word in words:
    word_counts[label][word] += 1  # Conta a ocorrência de cada palavra para a classe
    total_words[label] += 1  # Conta o total de palavras para cada classe

# Total de exemplos no corpus
total_examples = sum(class_counts.values())

# Probabilidade a priori (P(classe)) - probabilidade de uma classe ser escolhida sem observar o texto
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}

# Função para calcular a probabilidade condicional de uma palavra dada uma classe (P(palavra | classe))
# A suavização de Laplace é aplicada para evitar probabilidade zero (alpha=1)
def conditional_probability(word, label, alpha=1):
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo texto
# Função para prever a classe de um novo texto com base no cálculo das probabilidades.

# Função que recebe um texto, processa as palavras e calcula as probabilidades de cada classe
def predict(text):
  words = preprocess_text(text)  # Processa o novo texto
  probabilities = {}  # Dicionário para armazenar as probabilidades de cada classe

  # Calcula a probabilidade para cada classe
  for label in class_counts.keys():
    probabilities[label] = prior_probabilities[label]  # Inicializa com a probabilidade a priori da classe
    for word in words:
      probabilities[label] *= conditional_probability(word, label)  # Multiplica pelas probabilidades condicionais de cada palavra
  # Retorna a classe com a maior probabilidade e as probabilidades de cada classe
  return max(probabilities, key=probabilities.get), probabilities

# Passo 5 - Teste com um novo texto
# Aqui o modelo é testado com um novo texto para ver como ele classifica.

novo_texto = "Eu amo resolver bugs"  # Novo texto para classificar

# Chama a função `predict` para prever a classe e obter as probabilidades
classe, probs = predict(novo_texto)

# Exibe o texto, a classe prevista e as probabilidades
print(f'Texto: "{novo_texto}"')
print(f'Classe prevista: `{classe}`')
print(f'Probabilidades: {probs}')

[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo')]
Texto: "Eu amo resolver bugs"
Classe prevista: `positivo`
Probabilidades: {'positivo': 0.00040980807321904243, 'negativo': 0.0003048315805517451}


In [6]:
# Passo 1 - Importação das bibliotecas a serem utilizadas
# Importa o TfidfVectorizer para converter o texto em uma representação numérica utilizando o método TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Importa o modelo SVC (Support Vector Classifier) para classificação de textos
from sklearn.svm import SVC
# Importa a função para dividir os dados em conjuntos de treino e teste
from sklearn.model_selection import train_test_split
# Importa a função para gerar métricas de avaliação do modelo, como precisão, recall e f1-score
from sklearn.metrics import classification_report

# Passo 2 - Dados Exemplo
# Cria o corpus de textos (exemplo de documentos) que serão utilizados para treinamento e teste do modelo
corpus = [
    "Eu am PLN", "Eu odeio bugs", "Eu amo resolver problemas",
    "Odeio erros", "Amo programação", "Não gosto de falhas"
]
# Define os rótulos (classes) associados aos textos no corpus, indicando o sentimento (positivo ou negativo)
classes = ["negativo", "negativo", "positivo", "negativo", "positivo", "negativo"]

# Passo 3 - Pré-processamento e vetorização
# Cria o objeto TfidfVectorizer para converter o texto em uma representação numérica usando TF-IDF
vectorizer = TfidfVectorizer()
# Aplica o fit_transform no corpus, convertendo os textos para uma matriz esparsa TF-IDF
# X é a matriz de características (representação numérica dos textos) e y são as classes (rótulos)
X = vectorizer.fit_transform(corpus)
y = classes

# Passo 4 - Dividir os dados e treinar o modelo
# Divide os dados em conjuntos de treino e teste. 70% para treino e 30% para teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Cria o modelo de classificação utilizando Support Vector Machine (SVM) com um kernel linear
svm_model = SVC(kernel='linear')
# Treina o modelo SVM com os dados de treino
svm_model.fit(X_train, y_train)

# Passo 5 - Avaliar o modelo
# Faz a previsão das classes para o conjunto de teste
y_pred = svm_model.predict(X_test)
# Exibe o relatório de classificação, incluindo métricas como precisão, recall e f1-score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         2
    positivo       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# 1. Importar Bibliotecas
# Importa o NLTK (Natural Language Toolkit), utilizado para processamento de linguagem natural
import nltk
# Importa o TfidfVectorizer para representar o texto de forma numérica utilizando o método TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Importa a função para dividir os dados em treino e teste
from sklearn.model_selection import train_test_split
# Importa o classificador Naive Bayes para classificação de texto
from sklearn.naive_bayes import MultinomialNB
# Importa o classificador SVM (Support Vector Machine) para classificação de texto
from sklearn.svm import SVC
# Importa funções para avaliar o desempenho do modelo, como precisão, recall e f1-score
from sklearn.metrics import classification_report

# Baixar o dataset de exemplo de resenhas de filmes do NLTK
nltk.download('movie_reviews')
# Importa o dataset de resenhas de filmes do NLTK
from nltk.corpus import movie_reviews

# 2. Preparação dos dados

# Coleta de textos e suas classes (rótulos). As resenhas estão em dois rótulos: 'pos' (positivo) e 'neg' (negativo)
# Para cada categoria de resenha, a função movie_reviews.fileids retorna o ID de cada arquivo (resenha).
documents = [(" ".join(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()  # 'pos' e 'neg' são as categorias
             for fileid in movie_reviews.fileids(category)]  # Para cada arquivo dentro da categoria

# Separar os textos e os rótulos (sentimentos). texts contém as resenhas e labels contém as categorias ('pos' ou 'neg').
texts, labels = zip(*documents)

# Transformar rótulos (positivo/negativo) em 0 e 1 para poder trabalhar com os modelos de machine learning
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# A transformação mapeia 'pos' para 1 e 'neg' para 0
label = label_encoder.fit_transform(labels)

# Dividir os dados em treino (70%) e teste (30%) para avaliar o desempenho do modelo
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# 3. Representação do texto com TF-IDF
# Criar o vetorixador TF-IDF que será usado para converter os textos em vetores numéricos
# O parâmetro max_features=5000 limita a transformação às 5.000 palavras mais frequentes no corpus
vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar o vetorixador aos dados de treino e transformar os textos de treino em vetores
X_train = vectorizer.fit_transform(texts_train)
# Transformar os textos de teste em vetores utilizando o mesmo vetorixador ajustado aos dados de treino
X_test = vectorizer.transform(texts_test)

# 4. Treinar os modelos

# Treinamento do modelo Naive Bayes (MultinomialNB)
nb_model = MultinomialNB()
# Ajuste do modelo aos dados de treino
nb_model.fit(X_train, labels_train)

# Predição com o modelo Naive Bayes sobre o conjunto de teste
nb_predictions = nb_model.predict(X_test)

# Treinamento do modelo SVM (Support Vector Machine) com kernel linear
svm_model = SVC(kernel='linear')
# Ajuste do modelo SVM aos dados de treino
svm_model.fit(X_train, labels_train)

# Predição com o modelo SVM sobre o conjunto de teste
svm_predictions = svm_model.predict(X_test)

# 5. Avaliação

# Avaliação do modelo Naive Bayes
print("Naive Bayes Performance:")
# Exibe o relatório de avaliação para o modelo Naive Bayes: precisão, recall e F1-score
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))

# Avaliação do modelo SVM
print("SVM Performance:")
# Exibe o relatório de avaliação para o modelo SVM: precisão, recall e F1-score
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

